In [2]:
from chats import llm, creative_llm
from constants import article

from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [3]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate article titles."
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine
_____

{article}

_____
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.""",
input_variables=["article"]
)

In [6]:
print(user_prompt.format(article="Test String").content)

You are tasked with creating a name for a article.
The article is here for you to examine
_____

Test String

_____
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


In [11]:
from langchain_core.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

print(first_prompt.format(article="Test String"))

System: You are an AI assistant that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine
_____

Test String

_____
The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


In [12]:
chain_one = (
    {"article": lambda x: x["article"]}
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [13]:
article_title_msg = chain_one.invoke({"article": article})
article_title_msg

{'article_title': 'Neuro-Symbolic Agents: The Future of AI and the Road to AGI'}

In [15]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps builds good articles."
)
second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a description for
the article. The article is here for you to examine:

---

{article}

---

Here is the article title '{article_title}'.

Output the SEO friendly article description. Do not output
anything other than the description.""",
    input_variables=["article", "article_title"]
)

second_prompt = ChatPromptTemplate.from_messages([system_prompt, second_user_prompt])

In [19]:
chain_two = (
    {
        "article": lambda x: x["article"],
    "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summary": lambda x: x.content}
)

In [22]:
article_description_msg = chain_two.invoke({
    "article": article,
    "article_title": article_title_msg["article_title"]
})

In [23]:
article_description_msg

{'summary': "Discover how neuro-symbolic agents are shaping the future of AI and paving the way toward AGI. This article explores the fusion of neural AI (like LLMs) and symbolic computation, enabling AI to integrate with code, perform complex tasks, and access real-time data. Learn about the strengths of neural vs. symbolic systems, the MRKL architecture, and the ReAct framework that powers modern AI agents. Dive into the potential of neuro-symbolic systems to overcome LLM limitations, from lack of up-to-date knowledge to reasoning gaps. Explore advanced agent designs beyond tool calls, including multi-agent workflows and intent-based embeddings. Understand why agents are central to real-world AI applications and how they bridge the gap between current AI capabilities and the long-term vision of AGI. Whether you're a developer or AI enthusiast, this guide offers insights into the transformative role of neuro-symbolic agents in the AI landscape. Stay ahead with the latest on AI agents—

In [26]:
third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a new paragraph for the
article. The article is here for you to examine:

---

{article}

---

Choose one paragraph to review and edit. During your edit
ensure you provide constructive feedback to the user so they
can learn where to improve their own writing.""",
    input_variables=["article"]
)

# prompt template 3: creating a new paragraph for the article
third_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    third_user_prompt
])

In [28]:
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(description="The original paragraph")
    edited_paragraph: str = Field(description="The improved edited paragraph")
    feedback: str = Field(description="Constructive feedback on the original paragraph")


structured_llm = creative_llm.with_structured_output(Paragraph)

In [29]:
chain_three = (
    {"article": lambda x: x["article"]}
    | third_prompt
    | structured_llm
    | {
        "original_paragraph": lambda x: x.original_paragraph,
        "edited_paragraph": lambda x: x.edited_paragraph,
        "feedback": lambda x: x.feedback
    }
)

In [ ]:
out = chain_three.invoke({"article": article})
out